# DAT 19: Homework 4 Assignment - SVMs, Trees, RF

## Instructions

The goal of this homework is to review and bring together what we have learned about Support Vector Machines, Decision Trees, Random Forests, and ensembles. 

Please do all your analysis to answer the questions below in this Jupyter notebook. Show your work.

**Please submit your completed notebook by 6:30PM on Wednesday, February 17.**

## About the Data

Use the cancer_uci.csv dataset in the Data directory of our course repo. This is the [Breast Cancer Wisconsin](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Original)) dataset from the UCI ML Repository.

## Homework Assignment

**1) Load the data and check for balance between the two classes. If the ratio is less than 60/40 rebalance the classes to 50/50. We've provided some help here to get you started.**

In [1]:
import pandas as pd
import numpy as np

canc = pd.read_csv("cancer_uci.csv", index_col=0)
canc.head()
canc.Class.value_counts()
# What are the frequencies of each class?

Benign       458
Malignant    241
Name: Class, dtype: int64

We have imbalanced classes so we need to decide if we want to undersample, and take only 241 values from the Benign category, or oversample, and artificially inflate the volume of malignant data. First, let's convert to binary 1,0 for classification.

In [2]:
canc.Class = canc.Class.map({'Benign':0,'Malignant':1})
canc.Class.value_counts()

0    458
1    241
Name: Class, dtype: int64

To undersample, we would throw away almost half of our benign examples, which would greatly alter our dataset and we don't want to lose that much info. So let's oversample! Here is a pattern for how to oversample:

In [3]:
# Separate your two classes:
mal_example = canc[canc.Class == 1]
benign_example = canc[canc.Class == 0]

# Oversample the malignant class to have a 50/50 ratio:
mal_over_example = mal_example.sample(458,replace=True)

# Recombine the two frames:
over_sample = pd.concat([mal_over_example,benign_example])

# Sanity check the length:
print len(over_sample)

916


**2) Are the features normalized? If not, use the scikit-learn standard scaler to normalize them.**

In [4]:
over_sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 916 entries, 570 to 695
Data columns (total 11 columns):
Sample_code_number             916 non-null int64
Clump_Thickness                916 non-null int64
Uniformity_of_Cell_Size        916 non-null int64
Uniformity_of_Cell_Shape       916 non-null int64
Marginal_Adhesion              916 non-null int64
Single_Epithelial_Cell_Size    916 non-null int64
Bare_Nuclei                    916 non-null object
Bland_Chromatin                916 non-null int64
Normal_Nucleoli                916 non-null int64
Mitoses                        916 non-null int64
Class                          916 non-null int64
dtypes: int64(10), object(1)
memory usage: 85.9+ KB


In [5]:
over_sample.Bare_Nuclei[over_sample.Bare_Nuclei=='?'] = 0

/Users/bhambre/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [6]:
over_sample.Bare_Nuclei = over_sample.Bare_Nuclei.astype(float) 

In [7]:
over_sample.Bare_Nuclei[over_sample.Bare_Nuclei==0] = over_sample.Bare_Nuclei.mean()

/Users/bhambre/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [8]:
features = over_sample.drop(['Class','Sample_code_number'],axis=1)
target = over_sample.Class

In [9]:
#normalize data
from sklearn.preprocessing import StandardScaler
n_features = StandardScaler().fit_transform(features)
n_features = pd.DataFrame(features)

In [10]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split( n_features, target, test_size=0.20,random_state=0)

In [11]:
n_features.head()

,Clump_Thickness,Uniformity_of_Cell_Size,Uniformity_of_Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses
570,8,10,4,4,8,10,8,2,1
457,8,8,9,6,6,3,10,10,1
334,8,6,7,3,3,10,3,4,2
239,10,4,3,2,3,10,5,3,2
239,10,4,3,2,3,10,5,3,2


**3) Train a linear SVM, using the cross validated accuracy as the score (use the scikit-learn method).**

In [12]:
#your code here
from sklearn.cross_validation import cross_val_score
from sklearn.svm import SVC
model = SVC(kernel='linear',C=1).fit(X_train,y_train)

In [13]:
print cross_val_score(model,X_train,y_train,cv=5).mean()

0.975369522396


**4) Display the confusion matrix, classification report, and AUC.**

In [21]:
#your code here
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_train,model.predict(X_train))
print cm
print classification_report(y_test,model.predict(X_test))
#model_lr = LogisticRegression(C=1).fit(train_feat, train_target)
target_predicted_proba = model.predict_proba(X_test)
plot_roc_curve(y_test, target_predicted_proba)

[[347  12]
 [  1 372]]
             precision    recall  f1-score   support

          0       0.97      0.99      0.98        99
          1       0.99      0.96      0.98        85

avg / total       0.98      0.98      0.98       184



AttributeError: predict_proba is not available when probability=False

**5) Repeat steps 2 through 4 using a Decision Tree model. Are the results better or worse than the SVM?**

In [ ]:
#your code here

**6) Repeat steps 2 through 4 using a Random Forest model. Are the results better or worse than the SVM?**

In [ ]:
#your code here

### Extra Credit Questions
**The following questions are strongly encouraged, but not required for this homework assignment.**

**7) Combine the SVM and the Decision Tree model using the [Voting Classifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html). Are the results better than either of these base classifiers alone?**

In [ ]:
#your code here

**8) Train an SVM using the RBF kernel. Is this model better or worse?**

In [ ]:
#your code here